In [1]:
!pip install selenium

In [65]:
#자동 로그인
import selenium
from selenium import webdriver as wd
import time
import pandas as pd
import random

# 세종대 에브리타임 자동 로그인

options = wd.ChromeOptions()
options.add_argument("no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent={Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36}")

driver = wd.Chrome(executable_path='/usr/local/bin/chromedriver', options = options)
driver.maximize_window()


baseURL = 'https://sejong.everytime.kr'

rand_value = random.uniform(2,4)
time.sleep(rand_value)
driver.get(baseURL)

rand_value = random.uniform(2,4)
time.sleep(rand_value)
driver.find_element_by_css_selector('#account > a:nth-child(1)').click()

driver.find_element_by_name('userid').send_keys('gimsg12')
driver.find_element_by_name('password').send_keys('data')

rand_value = random.uniform(2,4)
time.sleep(rand_value)
driver.find_element_by_xpath('//*[@class="submit"]/input').click()

In [66]:
lecture = pd.read_csv("data/lecture_data.csv")
lecture.head()

,Unnamed: 0,url_id,lec_num,pro_name,star
0,0,535020,8802,이재교,86.6
1,1,1950200,9600,김희정,96.4
2,2,2099759,9608,한명관,100.0
3,3,1788599,3706,이원,95.0
4,4,1788598,9607,한명관,75.0


In [48]:
from bs4 import BeautifulSoup

In [67]:
url = "https://everytime.kr/lecture/view/"+url_id_list[0]
rand_value = random.uniform(2,5)
time.sleep(rand_value)
driver.get(url)
    
    #하나의 강의 평가 페이지 bs로 데이터 수집
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html,'html.parser') #정적텍스트 분석을 사용하는 BS이 selenium보다 빠름 -> 페이지 접근에는 동적 수집, 데이터 수집은 정적수집적
    
try:
    lecture_name = soup.select_one("#container > div.side.head > h2").text
    print(lecture_name)
    professor_name = soup.select_one("#container > div.side.head > p:nth-of-type(1) > span").text
    print(professor_name)
    review_list = soup.select("article")
    print(lecture_name, professor_name)
    print(review_list)
except: #왜 안되는지 모르게씀...;;;
        pass

법학입문
이재교
법학입문 이재교
[<article><a class="button abuse">신고</a><a class="button posvote">추천</a><p class="rate"><span class="star"><span class="on" style="width: 100%;"></span></span></p><p class="info"><span class="semester">20년 1학기 수강자</span></p><p class="text">판례평석 잘하면 성적잘 주시는것 같음. 중간 개망치고 기말도 틀린 문제 몇 개 있었는데 판례평석 과제 평가는 전부 A+이었음. 성적도 결국 A+ 나옴.</p></article>, <article><a class="button abuse">신고</a><a class="button posvote">추천</a><p class="rate"><span class="star"><span class="on" style="width: 80%;"></span></span></p><p class="info"><span class="semester">20년 1학기 수강자</span></p><p class="text">과제들 되게 귀찮고 많다 시험은 말빨 좋으면 된다 난 시험은 망쳤지만 과제는 열심히해서 점수 잘 받았음</p></article>, <article><a class="button abuse">신고</a><a class="button posvote">추천</a><p class="rate"><span class="star"><span class="on" style="width: 80%;"></span></span></p><p class="info"><span class="semester">20년 1학기 수강자</span></p><p class="text">수업은 ppt 띄워놓으시고 하시는데 보통 교안에 없는 다른 관련 사례를 중심으로 많이 얘기하심. 수업만 잘 들으면 시험도 잘 볼 수 있고 성적도 후하게 주시는 편. 다

In [78]:
import re
index = 0
#강의 -> 강의평가
review_data=[]
url_id_list = lecture.url_id.astype(str) 

for url_id in url_id_list[:]:
    #강의 페이지로 넘어가기
    url = "https://everytime.kr/lecture/view/"+url_id
    rand_value = random.uniform(2,5)
    time.sleep(rand_value)
    driver.get(url)
    
    #하나의 강의 평가 페이지 bs로 데이터 수집
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser') #정적텍스트 분석을 사용하는 BS이 selenium보다 빠름 -> 페이지접근에는 동적 수집, 데이터 수집은 정적수집적
    
    text = soup.select("div.articles > article > p.text")
    star = soup.select("div.articles > article > p.rate > span > span")
    info = soup.select("div.articles > article > p.info")

    table = list(zip(text, star, info))
    for index, value in enumerate(table[::-1]):
        text, star, info = value
        star = float(star.get("style").split()[1][:-2])
        semester = info.select_one("span.semester")
        print(semester)
        like = info.select_one("span.posvote")
        print(like)
        if like == None:
            like = 0
        else:
            like = int(like.get_text())
    
        review_data.append(
            {"url_id": url_id, "index": index, "star": star, "semester": semester.get_text(), "text": text.get_text(), "like": like}
        )

<span class="semester">20년 1학기 수강자</span>
None
<span class="semester">20년 1학기 수강자</span>
None
<span class="semester">19년 1학기 수강자</span>
<span class="posvote">4</span>
<span class="semester">20년 1학기 수강자</span>
None
<span class="semester">20년 1학기 수강자</span>
None
<span class="semester">20년 1학기 수강자</span>
None
<span class="semester">18년 1학기 수강자</span>
<span class="posvote">1</span>
<span class="semester">19년 1학기 수강자</span>
None
<span class="semester">19년 1학기 수강자</span>
None
<span class="semester">20년 1학기 수강자</span>
None
<span class="semester">19년 1학기 수강자</span>
<span class="posvote">1</span>
<span class="semester">19년 1학기 수강자</span>
None
<span class="semester">20년 1학기 수강자</span>
None
<span class="semester">20년 1학기 수강자</span>
<span class="posvote">2</span>
<span class="semester">19년 1학기 수강자</span>
None
<span class="semester">19년 1학기 수강자</span>
None
<span class="semester">19년 1학기 수강자</span>
None
<span class="semester">19년 1학기 수강자</span>
None
<span class="semester">19년 1학기 수강자</span>
None
<sp

In [79]:
review_data = pd.DataFrame(review_data)
review_data.to_csv('data/review_data.csv',index=False,encoding='utf-8-sig') #한글 인코딩 깨질때
len(review_data)

34

In [80]:
review_data

,url_id,index,star,semester,text,like
0,2099759,0,100.0,20년 1학기 수강자,학생들이 어떻게 하면 이해하기 쉬울까 고민하고 강의를 준비하시는 느낌입니다.교재도 ...,0
1,2099759,1,100.0,20년 1학기 수강자,조문 위주로 설명해주시며 특히 학생들이 어려워하는포인트를 정확하게 집어서 상세하게 ...,0
2,1788599,0,100.0,19년 1학기 수강자,"교수님 완전 친절하고 좋으심모든 공지, 교안파일, 과제, 시험점수를 메일로 보내주시...",4
3,1788599,1,100.0,20년 1학기 수강자,"내가 수업을 더 열심히 못들은게 죄송할 정도로, 비대면 강의라 수업을 더 잘 진행하...",0
4,1788599,2,80.0,20년 1학기 수강자,(온라인 기준)교수님 정말 열심히 하심과제도 엄청 쉽고 다 좋은데 시험이 정~~~~...,0
5,1788599,3,100.0,20년 1학기 수강자,교수님이 엄청 친절하고 좋으십니다. 공부해야할 양이 많긴 했는데 과제도 간단한 레포...,0
6,1788598,0,100.0,18년 1학기 수강자,18년부터 강의 시작하심. 검사장 출신으로 학구파 이지만 강의와 시험은 매우 Eas...,1
7,1788598,1,20.0,19년 1학기 수강자,생각보다 점수 잘 안줌. 과제 엄청 많음. 블랙보드에 강의자료 안떠서 안뜬다고 메일...,0
8,1788598,2,80.0,19년 1학기 수강자,생각보다 할만했음 강의력도 타 법학과교수님들에 비해 괜찮았고자료가 많다는 건 귀찮지...,0
9,1788598,3,100.0,20년 1학기 수강자,교수님 너무좋아요ㅠ 녹화강의인데도 세세하게 잘 가르쳐주시고.. 비대면인게 아쉬울정도ㅠㅠㅠ,0


In [50]:
review_data = pd.DataFrame(lecture_review_list)
review_data.to_csv('data/review_data.csv',index=False,encoding='utf-8-sig') #한글 인코딩 깨질때
len(review_data)

62